In [1]:
import numpy as np
import torch

A=np.arange(0,10)
B=np.arange(10,20)
A=A.reshape(10,1)
B=B.reshape(10,1)



C = np.hstack((A,B))
# C = torch.cat([A,B],dim=0) # 행방향으로 합침. 단 np가 torch로 바뀌어야 됨

print(C)

cond_domain1_heat_bc = np.matrix([0,1,2,3,4])
#id1 = domain1_point_all[cond_domain1_heat_bc,1] <=13.5 

id1 = C[cond_domain1_heat_bc,1] <=13.5 

print()
print(id1.shape)
print(cond_domain1_heat_bc.shape)

print(id1)


[[ 0 10]
 [ 1 11]
 [ 2 12]
 [ 3 13]
 [ 4 14]
 [ 5 15]
 [ 6 16]
 [ 7 17]
 [ 8 18]
 [ 9 19]]

(1, 5)
(1, 5)
[[ True  True  True  True False]]


In [2]:
help(torch.mean)

Help on built-in function mean in module torch:

mean(...)
    mean(input, *, dtype=None) -> Tensor
    
    Returns the mean value of all elements in the :attr:`input` tensor.
    
    Args:
        input (Tensor): the input tensor.
    
    Keyword args:
        dtype (:class:`torch.dtype`, optional): the desired data type of returned tensor.
            If specified, the input tensor is casted to :attr:`dtype` before the operation
            is performed. This is useful for preventing data type overflows. Default: None.
    
    Example::
    
        >>> a = torch.randn(1, 3)
        >>> a
        tensor([[ 0.2294, -0.5481,  1.3288]])
        >>> torch.mean(a)
        tensor(0.3367)
    
    .. function:: mean(input, dim, keepdim=False, *, dtype=None, out=None) -> Tensor
       :noindex:
    
    Returns the mean value of each row of the :attr:`input` tensor in the given
    dimension :attr:`dim`. If :attr:`dim` is a list of dimensions,
    reduce over all of them.
    
    
    I

In [2]:
#q1 = C[cond_domain1_heat_bc[id1],1] * 10/13.5
q1 = C[cond_domain1_heat_bc[id1],1]

print(q1)

[[10 11 12 13]]


In [3]:
# domain x, t를 만든다!
domain1_x = np.linspace(0, 2.5, 21)
domain1_t = np.arange(0, 27+0.1, 0.1)
x, t = np.meshgrid(domain1_x, domain1_t)
domain1_grid = np.hstack((x.flatten()[:,None], t.flatten()[:,None]))


In [4]:
print((x.flatten())[0:40].shape)
AAAA = (x.flatten())[0:40, None]
print(AAAA.shape)

#print((t.flatten())[0:40, None])

(40,)
(40, 1)


In [5]:
point_all = domain1_grid
indexes  = np.unique(point_all, axis=0, return_index=True)[1]   
point_all2 = point_all[np.sort(indexes),:]   
x = point_all[np.sort(indexes),:]   
print(point_all2.shape)
print(x.shape)

(5691, 2)
(5691, 2)


In [6]:
'''        point 구분        '''
domain1_point_x = torch.tensor(x[:,0].reshape(-1,1)).float()    # x 값만 다시 받아옴
domain1_point_t = torch.tensor(x[:,1].reshape(-1,1)).float()    # t 값만 다시 받아옴

# 240622 김영중 수정
domain1_point_all = torch.cat([domain1_point_x, domain1_point_t], dim=1)    # domain1_point_x와 domain1_point_t를 옆으로 쌓아? 그럼 다시 X 형태의 (5691,2) 인거 같은데?




In [7]:
# min-max 정규화를 이렇게 직접 해줬음
min_input_x = np.min(x[:,0])
max_input_x = np.max(x[:,0])

min_input_t = np.min(x[:,1])
max_input_t = np.max(x[:,1])

min_output = 40
max_output = 100

# 정규화 과정에서 붙음 coefficient 같은거
cc_xx = (max_output-min_output) / (max_input_x-min_input_x)**2
cc_x = (max_output-min_output) / (max_input_x-min_input_x)
cc_t = (max_output-min_output) / (max_input_t-min_input_t)

input_x_normalized = (x[:,0].copy() - min_input_x) / (max_input_x - min_input_x)
input_t_normalized = (x[:,1].copy() - min_input_t) / (max_input_t - min_input_t)

# 어쨌든 input 은 normalize 했다는 뜻
domain1_point_x_normal = torch.tensor(input_x_normalized.reshape(-1,1), requires_grad=True).float()  # domain 값을 normalized 시행
domain1_point_t_normal = torch.tensor(input_t_normalized.reshape(-1,1), requires_grad=True).float()  # domain 값을 normalized 시행


In [2]:
import numpy as np

help(np.hstack)

Help on _ArrayFunctionDispatcher in module numpy:

hstack(tup, *, dtype=None, casting='same_kind')
    Stack arrays in sequence horizontally (column wise).
    
    This is equivalent to concatenation along the second axis, except for 1-D
    arrays where it concatenates along the first axis. Rebuilds arrays divided
    by `hsplit`.
    
    This function makes most sense for arrays with up to 3 dimensions. For
    instance, for pixel-data with a height (first axis), width (second axis),
    and r/g/b channels (third axis). The functions `concatenate`, `stack` and
    `block` provide more general stacking and concatenation operations.
    
    Parameters
    ----------
    tup : sequence of ndarrays
        The arrays must have the same shape along all but the second axis,
        except 1-D arrays which can be any length.
    
    dtype : str or dtype
        If provided, the destination array will have this dtype. Cannot be
        provided together with `out`.
    
    .. versionadd

In [ ]:



'''        BC, IC index        '''
cond_domain1_heat_bc = ( (domain1_point_all[:,0]==0) ).nonzero(as_tuple=True)[0]    # 여기 자세히 들여다봐야될듯 어떻게 나누고 있을까?240609
cond_domain1_insul_bc = ( (domain1_point_all[:,0]==2.5)  ).nonzero(as_tuple=True)[0]
cond_domain1_T0_IC = ( (domain1_point_all[:,1]==0)  ).nonzero(as_tuple=True)[0]

In [14]:
kkkkk=np.sort(indexes)
print(kkkkk)

[   0    1    2 ... 5688 5689 5690]


In [11]:
# unique는 그 요소 중 겹치는거 빼고, basis는 몇개로 구성되어 있냐는 뜻이고 axis는 그 방향으로 봤을때!
# 아래 예제가 제대로 된 의미!

a = np.array([[1, 0, 0],
              [1, 0, 0],
              [2, 3, 4]])

print(np.unique(a, axis = 0, return_index=True))
print(np.unique(a, axis = 1, return_index=True))

(array([[1, 0, 0],
       [2, 3, 4]]), array([0, 2], dtype=int64))
(array([[0, 0, 1],
       [0, 0, 1],
       [3, 4, 2]]), array([1, 2, 0], dtype=int64))
